In [1]:
!pip install langchain langchain_pinecone langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Un

In [2]:
from langchain import PromptTemplate
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vecteorstores import Pinecone 
from pinecone import Pinecone,ServerlessSpec
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os
from tqdm.autonotebook import tqdm
import sys
sys.path.append('../../')

In [3]:
PINECONE_INDEX_NAME = "medical-chatbot"

In [4]:
os.environ['PINECONE_API_KEY'] = "pcsk_4sFyzU_Gih859tfFJTQQN95awxTFyjNStq7uHh6JSGzUVTbjaXh6qWh5BbActVN3NnUNYC"

In [19]:
import os
from pathlib import Path
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

def load_data(directory_path):
    dir_path = Path(directory_path)

    # Vérifier si le dossier existe
    if not dir_path.exists() or not dir_path.is_dir():
        raise FileNotFoundError(f"Le dossier '{directory_path}' n'existe pas ou n'est pas un répertoire.")

    # Vérifier s'il y a des PDFs
    pdf_files = list(dir_path.glob("*.pdf"))
    if not pdf_files:
        raise FileNotFoundError(f"Aucun fichier PDF trouvé dans '{directory_path}'.")

    # Charger les PDFs
    loader = DirectoryLoader(directory_path, glob="*.pdf", loader_cls=PyPDFLoader)
    data = loader.load()
    
    print(f"\n✅ {len(data)} pages chargées depuis {len(pdf_files)} fichiers PDF.")

    # Afficher le contenu de toutes les pages avec leur fichier d'origine
    for i, doc in enumerate(data):
        print(f"\n📄 Fichier : {doc.metadata.get('source', 'Inconnu')}")
        print(f"--- Page {i+1} ---\n")
        print(doc.page_content[:1000])  # Afficher un extrait
    
    return data

# Spécifier le répertoire contenant les PDFs
directory_path = "/kaggle/input/pdf-challenge/"

try:
    data = load_data(directory_path)
except FileNotFoundError as e:
    print(f"❌ Erreur : {e}")

e 3. Physiopathologie 
3.1. Mécanismes conduisant à l'irruption d'air dans l'espace pleural 
L'irruption d'air dans la cavité pleurale est la conséquence d'une brèche: 
• Soit au travers de la plèvre viscérale (PNO spontané, primitif ou secondaire)
Item 360 
L'air des voies aériennes s'infiltre à travers une lésion parenchymateuse. L'entrée d'air dans l'espace pleural via la 
plèvre viscérale résulte soit de la rupture d'alvéoles périphériques, de blebs, ou de bulles d'emphysème. Plus rarement, 
il peut s'agir de la rupture d'un processus nécrosant du parenchyme pulmonaire (abcès, caverne tuberculeuse, ou 
cancer, ou de lésions par blast). 
• Soit au travers de la plèvre pariétale (PNO traumatique)
L'entrée d'air dans l'espace pleural résulte soit d'une lésion directe (plaie par arme, ponction, biopsie, fracture 
costale), soit par un traumatisme non pénétrant (contusion thoracique). D'autres lésions thoraciques et/ou extra­
thoraciques peuvent être associées. 
Les PNO traumatiques son

In [20]:
def text_split(data):
    splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap= 20)
    text_chunks = splitter.split_documents(data)
    return text_chunks

In [21]:
text_chunks = text_split(data)
print(len(text_chunks))

34190


In [22]:
def download_huggingface_embedding():
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [23]:
embeddings = download_huggingface_embedding()

<ipython-input-22-2975e5bcb07e>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
vectorstore_from_docs = PineconeVectorStore.from_documents(
        text_chunks,
        index_name=PINECONE_INDEX_NAME,
        embedding=embeddings
    )

In [30]:
groq_api_key="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"

In [32]:
!pip install langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.0 MB/s eta 0:00:00 0:00:01


### Original code

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

# 1. Initialize Pinecone Vector Store
vectorstore = PineconeVectorStore.from_documents(
    text_chunks,
    index_name=PINECONE_INDEX_NAME,
    embedding=embeddings
)

# 2. Initialize LLM (Groq API - Llama 3.1 70B)
llm = ChatGroq(
    temperature=0,
    groq_api_key=groq_api_key,
    model_name="llama-3.3-70b-versatile"
)

# 3. Define the Prompt Template
prompt_template = PromptTemplate(
    template="""
    Given the following summarized conversation between doctors:  
    "{conversation_summary}"  
    
    Based on relevant medical documents, define the patient's case, suggest the medical process, and provide detailed guidance on preparing the treatment environment for receiving the patient from the ambulance.  
    
    Your response should include:  
    - **Diagnosis or potential medical condition.**  
    - **Steps for further medical investigation.**  
    - **Required equipment, medications, and staff preparation.**  
    - **Specific protocols for receiving the patient from the ambulance, including:**  
      - **Immediate stabilization procedures.**  
      - **Preparation of the emergency treatment area (e.g., ICU, trauma room).**  
      - **Coordination with paramedics for seamless handover.**  
      - **Emergency response team roles and readiness.**  
      - **Infection control or isolation measures if necessary.**  
    
    Use the retrieved documents to support the response.  
    """,
    input_variables=["conversation_summary"]
)

# 4. Create Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectorstore.as_retriever(),
    condense_question_prompt=prompt_template,
    return_source_documents=True
)

# 5. Test the RAG System
conversation_summary = "Patient exhibits symptoms of severe chest pain and shortness of breath, potentially cardiac-related. Discussed possible causes including angina, heart attack, or pulmonary embolism."

# ✅ Use `invoke()` with `chat_history`
response = qa_chain.invoke({
    "question": conversation_summary,
    "chat_history": []  # Empty list for first query
})

# 6. Print the result
print("Answer:", response["answer"])
print("Retrieved Documents:", response["source_documents"])

## Test With LLama 3.3 

In [45]:
from langchain.vectorstores import Pinecone
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

# 1. Initialize Pinecone Vector Store
vectorstore = PineconeVectorStore.from_documents(
    text_chunks,
    index_name=PINECONE_INDEX_NAME,
    embedding=embeddings
)

# 2. Initialize LLM (Groq API - Llama 3.1 70B)
llm = ChatGroq(
    temperature=0,
    groq_api_key=groq_api_key,
    model_name="llama-3.3-70b-versatile"
)

# 3. Define the Prompt Template
prompt_template = PromptTemplate(
    template="""
    Given the following doctors' notes on a patient consultation:  
    "{conversation_summary}"  
    
    Based on relevant medical documents, define the patient's case, suggest the medical process, and provide detailed guidance on preparing the treatment environment for receiving the patient from the ambulance.  
    
    Your response should include:  
    - **Diagnosis or potential medical condition.**  
    - **Steps for further medical investigation.**  
    - **Required equipment, medications, and staff preparation.**  
    - **Specific protocols for receiving the patient from the ambulance, including:**  
      - **Immediate stabilization procedures.**  
      - **Preparation of the emergency treatment area (e.g., ICU, trauma room).**  
      - **Coordination with paramedics for seamless handover.**  
      - **Emergency response team roles and readiness.**  
      - **Infection control or isolation measures if necessary.**  
    
    Use the retrieved documents to support the response.  
    """,
    input_variables=["conversation_summary"]
)

# 4. Define the chain with the correct prompt template
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectorstore.as_retriever(),
    condense_question_prompt=prompt_template,  # Ensure the correct prompt template is used
    return_source_documents=True
)

# 5. Test the RAG System (Provide full prompt instead of just the summary)
conversation_summary = """The patient is a 26-year-old female, referred to Physical Therapy for low back pain.
The patient has a history of traumatic injury to low back. The patient stated initial injury occurred eight years ago,
when she fell at a ABC Store. The patient stated she received physical therapy, one to two visits and received modality 
treatment only, specifically electrical stimulation and heat pack per patient recollection. The patient stated that she 
has had continuous low-back pain at varying degrees for the past eight years since that fall. The patient gave birth in 
August 2008 and since the childbirth, has experienced low back pain. The patient also states that she fell four to five 
days ago, while mopping her floor. The patient stated that she landed on her tailbone and symptoms have increased since 
that fall. The patient stated that her initial physician examination with Dr. X was on 01/10/09, and has a followup 
appointment on 02/10/09."""

# ✅ Use `invoke()` with `chat_history`
response = qa_chain.invoke({
    "question": prompt_template.format(conversation_summary=conversation_summary),
    "chat_history": []  # Empty list for first query
})

# 6. Print the result
print("Answer:", response["answer"])
print("Retrieved Documents:", response["source_documents"])


Answer: Based on the provided doctor's notes, the patient's case can be defined as a 26-year-old female with a history of traumatic injury to her low back, resulting in continuous low-back pain for the past eight years. The patient's condition has been exacerbated by a recent fall, which has increased her symptoms.

**Diagnosis or potential medical condition:**
The patient's potential medical condition is chronic low back pain, possibly with underlying conditions such as herniated discs, spinal stenosis, or spondylosis. The recent fall may have also caused acute injuries such as coccydynia (tailbone pain) or lumbar strain.

**Steps for further medical investigation:**
To further investigate the patient's condition, the following steps can be taken:

1. Conduct a thorough physical examination, including a neurological examination to assess for any signs of nerve damage or compression.
2. Order imaging studies such as X-rays, CT scans, or MRI scans to evaluate the patient's spinal anatom

## Test with deepseek

In [47]:
from langchain.vectorstores import Pinecone
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

# 1. Initialize Pinecone Vector Store
vectorstore = PineconeVectorStore.from_documents(
    text_chunks,
    index_name=PINECONE_INDEX_NAME,
    embedding=embeddings
)

# 2. Initialize LLM (Groq API - deepseek-r1-distill-qwen-32b)
llm = ChatGroq(
    temperature=0,
    groq_api_key=groq_api_key,
    model_name="deepseek-r1-distill-qwen-32b"
)

# 3. Define the Prompt Template
prompt_template = PromptTemplate(
    template="""
    Given the following doctors' notes on a patient consultation:  
    "{conversation_summary}"  
    
    Based on relevant medical documents, define the patient's case, suggest the medical process, and provide detailed guidance on preparing the treatment environment for receiving the patient from the ambulance.  
    
    Your response should include:  
    - **Diagnosis or potential medical condition.**  
    - **Steps for further medical investigation.**  
    - **Required equipment, medications, and staff preparation.**  
    - **Specific protocols for receiving the patient from the ambulance, including:**  
      - **Immediate stabilization procedures.**  
      - **Preparation of the emergency treatment area (e.g., ICU, trauma room).**  
      - **Coordination with paramedics for seamless handover.**  
      - **Emergency response team roles and readiness.**  
      - **Infection control or isolation measures if necessary.**  
    
    Use the retrieved documents to support the response.  
    """,
    input_variables=["conversation_summary"]
)

# 4. Define the chain with the correct prompt template
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectorstore.as_retriever(),
    condense_question_prompt=prompt_template,  # Ensure the correct prompt template is used
    return_source_documents=True
)

# 5. Test the RAG System (Provide full prompt instead of just the summary)
conversation_summary = """The patient is a 26-year-old female, referred to Physical Therapy for low back pain.
The patient has a history of traumatic injury to low back. The patient stated initial injury occurred eight years ago,
when she fell at a ABC Store. The patient stated she received physical therapy, one to two visits and received modality 
treatment only, specifically electrical stimulation and heat pack per patient recollection. The patient stated that she 
has had continuous low-back pain at varying degrees for the past eight years since that fall. The patient gave birth in 
August 2008 and since the childbirth, has experienced low back pain. The patient also states that she fell four to five 
days ago, while mopping her floor. The patient stated that she landed on her tailbone and symptoms have increased since 
that fall. The patient stated that her initial physician examination with Dr. X was on 01/10/09, and has a followup 
appointment on 02/10/09."""

# ✅ Use `invoke()` with `chat_history`
response = qa_chain.invoke({
    "question": prompt_template.format(conversation_summary=conversation_summary),
    "chat_history": []  # Empty list for first query
})

# 6. Print the result
print("Answer:", response["answer"])


Answer: <think>
Okay, so I need to figure out how to help this user. They provided a query where a user is asking about a patient's case, medical process, and treatment environment preparation. The user also included some context from medical documents, but it seems like the same document was repeated multiple times. 

First, I should read through the query carefully. The user wants a detailed response that includes diagnosis, steps for further investigation, required equipment, medications, staff preparation, and specific protocols for receiving the patient from the ambulance. They also mentioned using the retrieved documents to support the response.

Looking at the context provided, the documents are about low back pain, neurofibromatosis, self-reported height, and total knee replacement. The patient in question has a history of low back pain from a fall, childbirth, and a recent fall. So, the main issue seems to be chronic low back pain with recent exacerbation.

I need to structure

In [60]:
from langchain.vectorstores import Pinecone
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

# 1. Initialize Pinecone Vector Store
vectorstore = PineconeVectorStore.from_documents(
    text_chunks,
    index_name=PINECONE_INDEX_NAME,
    embedding=embeddings
)

# 2. Initialize LLM (Groq API - Llama 3.1 70B)
llm = ChatGroq(
    temperature=0,
    groq_api_key=groq_api_key,
    model_name="llama-3.2-90b-vision-preview"
)

# 3. Define the Prompt Template
prompt_template = PromptTemplate(
    template="""
    Given the following doctors' notes on a patient consultation:  
    "{conversation_summary}"  
    
    Based on relevant medical documents, define the patient's case, suggest the medical process, and provide detailed guidance on preparing the treatment environment for receiving the patient from the ambulance.  
    
    Your response should include:  
    - **Diagnosis or potential medical condition.**  
    - **Steps for further medical investigation.**  
    - **Required equipment, medications, and staff preparation.**  
    - **Specific protocols for receiving the patient from the ambulance, including:**  
      - **Immediate stabilization procedures.**  
      - **Preparation of the emergency treatment area (e.g., ICU, trauma room).**  
      - **Coordination with paramedics for seamless handover.**  
      - **Emergency response team roles and readiness.**  
      - **Infection control or isolation measures if necessary.**  
    
    Use the retrieved documents to support the response.  
    """,
    input_variables=["conversation_summary"]
)

# 4. Define the chain with the correct prompt template
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectorstore.as_retriever(),
    condense_question_prompt=prompt_template,  # Ensure the correct prompt template is used
    return_source_documents=True
)

# 5. Test the RAG System (Provide full prompt instead of just the summary)
conversation_summary = """The patient is a 26-year-old female, referred to Physical Therapy for low back pain.
The patient has a history of traumatic injury to low back. The patient stated initial injury occurred eight years ago,
when she fell at a ABC Store. The patient stated she received physical therapy, one to two visits and received modality 
treatment only, specifically electrical stimulation and heat pack per patient recollection. The patient stated that she 
has had continuous low-back pain at varying degrees for the past eight years since that fall. The patient gave birth in 
August 2008 and since the childbirth, has experienced low back pain. The patient also states that she fell four to five 
days ago, while mopping her floor. The patient stated that she landed on her tailbone and symptoms have increased since 
that fall. The patient stated that her initial physician examination with Dr. X was on 01/10/09, and has a followup 
appointment on 02/10/09."""

# ✅ Use `invoke()` with `chat_history`
response = qa_chain.invoke({
    "question": prompt_template.format(conversation_summary=conversation_summary),
    "chat_history": []  # Empty list for first query
})

# 6. Print the result
print("Answer:", response["answer"])


Answer: **Diagnosis or potential medical condition:**
Based on the patient's history and symptoms, the potential medical condition is chronic low back pain, possibly exacerbated by the recent fall and childbirth. The patient's history of traumatic injury to the low back and continuous pain for eight years suggests a possible underlying condition such as a herniated disc, spinal stenosis, or spondylosis. The recent fall may have caused a new injury or exacerbated the existing condition.

**Steps for further medical investigation:**
To confirm the diagnosis, the following steps may be taken:

1. Physical examination: A thorough physical examination to assess the patient's range of motion, muscle strength, and reflexes.
2. Imaging studies: X-rays, CT scans, or MRI scans to evaluate the spine and surrounding tissues.
3. Electromyography (EMG) or nerve conduction studies (NCS) to assess muscle and nerve function.

**Required equipment, medications, and staff preparation:**
To prepare for th